In [13]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.sych.data_fc_db_raw import DataFCDatabase
import lib.analysis.coactivity as coactivity

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/pub-2020-exploratory-analysis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
# params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yaro/neuronal-raw-pooled'
# params['root_path_data'] = gui_fpath('h5path', './')

In [15]:
dataDB = DataFCDatabase(params)

Searching for data files
Extracting trial type names
Extracting data types


In [16]:
ds = DataStorage('sych_result_activity.h5')

In [17]:
mc = MetricCalculator(serial=True, verbose=False)

In [18]:
print(dataDB.mice)
print(dataDB.dataTypes)
print(dataDB.trialTypeNames)

{'mvg_9', 'mvg_7', 'mvg_8', 'mvg_4'}
{'bn_trial', 'bn_session', 'raw'}
{'iGO', 'iFA', 'iNOGO', 'iMISS'}


# 1. Significance

## 1.1. Correlation plots


In [20]:
coactivity.corr_plot_session_composite(dataDB, mc, 'corr',
                                       dataTypes=['bn_session', 'bn_trial'],
                                       trialTypes=[None, 'iGO', 'iNOGO'],
                                       performances=['naive', 'expert'],
                                       haveMono=False, dropChannels=[21])

mvg_9_PRE_bn_session_None_naive
0 : 0.6896207987717927 0.19863891432013392 : ['Pir' 'AOL' 'AOD' 'M2' 'Cg1' 'PrL' 'MO' 'V1_m' 'RSG']
1 : 0.6711651626632565 0.20102785814254875 : ['AI' 'M1_m' 'IV2' 'S' 'CA1_Mol' 'S1_bf' 'Cpu' 'BLA' 'S1_bf_1' 'BMA' 'S2'
 'Cpu_1' 'LGP']
2 : 0.7394551929049803 0.2803246459032364 : ['BIC' 'SubB' 'InWh' 'SuG' 'LPAG' 'Rt' 'iGP' 'VPM' 'CA1_Py' 'VM' 'dCA1'
 'LaDL' 'GP' 'VPL' 'LDVL' 'VL' 'DG_a']
3 : 0.479197840099841 0.14212908265940027 : ['M1_l' 'LO' 'VO' 'DG_p' 'Sub' 'eGP' 'Po' 'M2_1']
mvg_9_PRE_bn_session_None_expert
0 : 0.4196316315744288 0.004820381729681578 : ['M1_l' 'M1_m' 'Pir' 'LO' 'AOL' 'VO' 'DG_p' 'S1_bf' 'Sub' 'Cpu_1' 'eGP'
 'Po' 'M2_1']
1 : 0.7024016161085597 0.22524241625725588 : ['AOD' 'M2' 'Cg1' 'PrL' 'MO' 'BIC' 'SubB' 'RSG' 'InWh' 'SuG' 'LPAG' 'iGP'
 'VPM' 'CA1_Py' 'VM' 'dCA1' 'LaDL' 'VPL' 'LDVL' 'VL' 'DG_a']
2 : 0.7929001081996134 0.23867809376373972 : ['AI' 'IV2' 'S' 'CA1_Mol' 'V1_m' 'Cpu' 'BLA' 'S1_bf_1' 'BMA' 'Rt' 'S2'
 'LGP' 'GP']
mvg_9_PRE_

/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 : nan nan : ['VO']
1 : 0.8839786663996139 0.5080882741576884 : ['AI' 'M1_m' 'Pir' 'AOL' 'AOD' 'M2' 'IV2' 'S' 'CA1_Mol' 'BIC' 'SubB'
 'InWh' 'SuG' 'LPAG' 'Cpu' 'BLA' 'S1_bf_1' 'BMA' 'Rt' 'iGP' 'VPM' 'CA1_Py'
 'VM' 'dCA1' 'S2' 'LaDL' 'LGP' 'GP' 'VPL' 'LDVL' 'VL' 'DG_a']
2 : 0.6262053684331153 0.043610370861032255 : ['M1_l' 'Cg1' 'PrL' 'MO' 'DG_p' 'V1_m' 'RSG' 'S1_bf' 'Cpu_1']
3 : nan nan : ['eGP']
4 : 0.6497723552005982 0.3704221082467004 : ['LO' 'Sub' 'Po' 'M2_1']
mvg_9_PRE_bn_trial_None_expert


/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 : 0.8465202911383692 0.37676442799775633 : ['M1_l' 'AI' 'Pir' 'AOD' 'SuG' 'LaDL' 'GP' 'M2_1']
1 : 0.774388253807202 0.4262318219704707 : ['LO' 'VO' 'V1_m' 'LGP']
2 : 0.6946278369785034 0.5222510192598152 : ['CA1_Mol' 'DG_p' 'S1_bf']
3 : 0.8546891092424178 0.4022789993094556 : ['M2' 'Cg1' 'PrL' 'MO' 'IV2' 'S' 'BIC' 'SubB' 'InWh' 'LPAG' 'Cpu' 'Rt'
 'iGP' 'VPM' 'CA1_Py' 'dCA1' 'VPL' 'LDVL' 'VL' 'DG_a']
4 : 0.8460367576352125 0.2171066014694385 : ['M1_m' 'AOL' 'RSG' 'BLA' 'S1_bf_1' 'BMA' 'S2']
5 : nan nan : ['eGP']
6 : 0.8171827100915913 0.49579498483843476 : ['VM' 'Sub' 'Cpu_1' 'Po']
mvg_9_PRE_bn_trial_iGO_naive


...

/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 : 0.9191263585410471 0.4930310802442098 : ['M1_l' 'AI' 'Pir' 'LO' 'AOL' 'AOD' 'M2' 'VO' 'Cg1' 'PrL' 'MO' 'CA1_Mol'
 'InWh' 'LPAG' 'S1_bf' 'S1_bf_1' 'Rt' 'iGP' 'VPM' 'CA1_Py' 'VM' 'dCA1'
 'Sub' 'LaDL' 'Cpu_1' 'eGP' 'GP' 'VPL' 'LDVL' 'Po' 'VL' 'M2_1' 'DG_a']
1 : 0.586540585172402 0.17938142336489088 : ['M1_m' 'V1_m' 'SuG']
2 : 0.7531717382352975 0.23195041098891078 : ['IV2' 'S' 'DG_p' 'BIC' 'SubB' 'LGP']
3 : 0.9411575845936219 0.5258026777176401 : ['RSG' 'Cpu' 'S2']
4 : nan nan : ['BLA']
5 : nan nan : ['BMA']
mvg_4_PRE_bn_trial_iGO_expert
0 : 0.8045563516206401 0.4257170681441249 : ['M1_l' 'Pir' 'PrL' 'S1_bf_1' 'LaDL' 'DG_a']
1 : 0.8279041246456438 0.5335354989069587 : ['AI' 'M1_m' 'LO' 'AOD' 'M2' 'VO' 'Cg1' 'InWh' 'SuG' 'Sub']
2 : 0.7685227650127722 0.5506762138419882 : ['AOL' 'IV2' 'S']
3 : 0.8848470073301756 0.8170820931520235 : ['V1_m' 'RSG' 'LPAG']
4 : 0.6976118856870299 0.628134948197939 : ['DG_p' 'BIC' 'S1_bf' 'BLA' 'BMA']
5 : 0.6254355077769965 0.5626533739242147 : ['SubB' 'Cpu

/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 : 0.7963414419041968 0.3889967387599225 : ['M1_m' 'SuG']
1 : 0.7025101137016573 0.1375511690922756 : ['IV2' 'S' 'DG_p' 'V1_m' 'BIC' 'SubB' 'RSG' 'Cpu' 'S2']
2 : nan nan : ['BLA']
3 : nan nan : ['BMA']
4 : 0.9410052407185148 0.4575487019026112 : ['M1_l' 'AI' 'Pir' 'LO' 'AOL' 'AOD' 'M2' 'VO' 'Cg1' 'PrL' 'MO' 'CA1_Mol'
 'InWh' 'LPAG' 'S1_bf' 'S1_bf_1' 'Rt' 'iGP' 'VPM' 'CA1_Py' 'VM' 'dCA1'
 'Sub' 'LaDL' 'Cpu_1' 'LGP' 'eGP' 'GP' 'VPL' 'LDVL' 'Po' 'VL' 'M2_1'
 'DG_a']
mvg_4_PRE_bn_trial_iNOGO_expert
0 : 0.8053628788887333 0.4214176198645123 : ['M1_l' 'Pir' 'PrL' 'S1_bf_1' 'LaDL' 'DG_a']
1 : 0.8104853382450107 0.5199585373764305 : ['AI' 'M1_m' 'LO' 'AOD' 'M2' 'VO' 'Cg1' 'InWh' 'SuG' 'Sub']
2 : 0.7865808328851568 0.5497528258795392 : ['AOL' 'IV2' 'S']
3 : 0.8852265374219321 0.8294637718548123 : ['V1_m' 'RSG' 'LPAG']
4 : 0.718689893879901 0.6287661640708629 : ['DG_p' 'BIC' 'S1_bf' 'BLA' 'BMA']
5 : 0.6119841064436099 0.5811391981870084 : ['SubB' 'Cpu' 'S2' 'Cpu_1']
6 : 0.8511023211336555 0.524

0 : 0.659644585666445 0.31955449564302735 : ['AI' 'M1_m' 'LO' 'AOL' 'AOD' 'M2' 'Cg1' 'InWh' 'LPAG' 'BLA' 'S1_bf_1'
 'Sub' 'LaDL']
1 : 0.5134177912653028 0.15305069501949553 : ['M1_l' 'Pir' 'VO' 'PrL' 'MO' 'IV2' 'BIC' 'S1_bf' 'BMA' 'Rt' 'iGP' 'VPM'
 'CA1_Py' 'VM' 'LGP' 'eGP' 'GP' 'VPL' 'Po' 'VL' 'M2_1' 'DG_a']
2 : 0.2182914471853042 -0.016554149747118857 : ['S' 'CA1_Mol' 'DG_p' 'V1_m' 'SubB' 'RSG' 'SuG' 'Cpu' 'dCA1' 'S2' 'Cpu_1'
 'LDVL']
mvg_4_REW_bn_session_iNOGO_naive
0 : 0.5639081823595664 0.39177637107973573 : ['M1_l' 'AI' 'Pir' 'LO' 'AOL' 'AOD' 'M2' 'VO' 'Cg1' 'PrL' 'MO' 'DG_p'
 'BIC' 'InWh' 'LPAG' 'S1_bf' 'Cpu' 'S1_bf_1' 'Rt' 'iGP' 'VPM' 'CA1_Py'
 'VM' 'Sub' 'LaDL' 'Cpu_1' 'LGP' 'eGP' 'GP' 'VPL' 'LDVL' 'Po' 'VL' 'M2_1'
 'DG_a']
1 : 0.06035741750416445 -0.11177439377566623 : ['M1_m' 'IV2' 'S' 'CA1_Mol' 'V1_m' 'SubB' 'RSG' 'SuG' 'BLA' 'BMA' 'dCA1'
 'S2']
mvg_4_REW_bn_session_iNOGO_expert
0 : 0.2607319609555345 -0.0614166410039258 : ['CA1_Mol' 'DG_p' 'V1_m' 'SubB' 'RSG' 'SuG' 'Cpu' '

**Drop first PCA and explore result**

In [22]:
coactivity.corr_plot_session_composite(dataDB, mc, 'corr',
                                       dataTypes=['bn_session', 'bn_trial'],
                                       trialTypes=[None, 'iGO', 'iNOGO'],
                                       performances=['naive', 'expert'],
                                       haveMono=False,
                                       nDropPCA=1, dropChannels=[21])

mvg_9_PRE_bn_session_None_naive
0 : 0.4754878318990574 0.6210580625478245 : ['AOL' 'AOD' 'M2' 'VO' 'Cg1' 'PrL' 'MO' 'RSG']
1 : 0.4366427229653429 0.517848307942502 : ['SubB' 'InWh' 'SuG' 'LPAG' 'CA1_Py' 'VPL']
2 : 0.40672816375329407 0.45814953245788353 : ['IV2' 'DG_p' 'S1_bf' 'Cpu_1']
3 : 0.39363030712194613 0.4937610970338651 : ['AI' 'M1_m' 'S' 'CA1_Mol' 'Cpu' 'BLA' 'S1_bf_1' 'BMA' 'S2' 'LGP']
4 : 0.3748007385175677 0.47562664832600837 : ['M1_l' 'Pir' 'LO' 'V1_m' 'Sub' 'eGP' 'Po' 'M2_1']
5 : 0.40678236105498994 0.5194673578325224 : ['BIC' 'Rt' 'iGP' 'VPM' 'VM' 'dCA1' 'LaDL' 'GP' 'LDVL' 'VL' 'DG_a']
mvg_9_PRE_bn_session_None_expert
0 : 0.34245527293329575 0.4546509561040554 : ['M1_l' 'Pir' 'LO' 'S1_bf' 'Cpu_1' 'eGP']
1 : 0.4286456117553373 0.528668640998515 : ['AOD' 'M2' 'VO' 'Cg1' 'PrL' 'MO' 'RSG']
2 : 0.4520810893634141 0.6260133967416244 : ['AI' 'M1_m' 'IV2' 'S' 'CA1_Mol' 'DG_p' 'Cpu' 'BLA' 'S1_bf_1' 'BMA' 'S2']
3 : 0.17328360704285492 0.1833290725880205 : ['BIC' 'SubB' 'Rt' 'VPM' 

0 : 0.3268923910840239 0.4182245008698973 : ['M1_l' 'Pir' 'LO' 'Cpu_1' 'eGP']
1 : 0.4880998177939068 0.5951694031969881 : ['AOD' 'M2' 'VO' 'Cg1' 'PrL' 'MO']
2 : 0.4253055473161463 0.5792848557887753 : ['AI' 'M1_m' 'IV2' 'S' 'CA1_Mol' 'DG_p' 'S1_bf' 'BLA' 'S1_bf_1' 'BMA' 'S2']
3 : 0.10273488673477711 0.12239302349850446 : ['V1_m' 'BIC' 'SubB' 'SuG' 'Cpu' 'Rt' 'LaDL' 'LGP' 'GP']
4 : 0.450564592354585 0.5597365876979314 : ['InWh' 'LPAG' 'iGP' 'VPM' 'CA1_Py' 'VM' 'dCA1' 'VPL' 'LDVL' 'VL' 'DG_a']
5 : 0.11316022190560215 0.15125997934180235 : ['AOL' 'RSG' 'Sub' 'Po' 'M2_1']
mvg_9_TEX_bn_session_iGO_naive
0 : 0.4338578555495128 0.5824194842157284 : ['AOL' 'AOD' 'M2' 'VO' 'Cg1' 'PrL' 'MO' 'RSG']
1 : 0.24097217334821439 0.33009639849077704 : ['M1_l' 'AI' 'M1_m' 'IV2' 'DG_p' 'S1_bf' 'Cpu_1' 'eGP']
2 : 0.36354683363027174 0.4366615214930399 : ['S' 'CA1_Mol' 'Cpu' 'BLA' 'S1_bf_1' 'BMA' 'S2' 'LaDL' 'LGP']
3 : 0.40630145164445686 0.55185906571776 : ['BIC' 'SubB' 'InWh' 'LPAG' 'Rt' 'iGP' 'VPM' 'CA1_P

0 : 0.4395326123911103 0.5219770384446083 : ['AOD' 'M2' 'VO' 'Cg1' 'PrL' 'MO']
1 : 0.38331421904476215 0.517697181484973 : ['AI' 'M1_m' 'S' 'CA1_Mol' 'DG_p' 'Cpu' 'BLA' 'S1_bf_1' 'BMA' 'S2']
2 : 0.03479264227670027 0.05617361666606732 : ['AOL' 'BIC' 'SubB' 'SuG' 'LPAG' 'Rt' 'LaDL' 'LGP' 'GP' 'VPL' 'Po' 'M2_1']
3 : 0.1588796746169696 0.23432807654468935 : ['M1_l' 'Pir' 'LO' 'IV2' 'V1_m' 'S1_bf' 'Cpu_1' 'eGP']
4 : 0.37312929217242674 0.4765678179679856 : ['RSG' 'InWh' 'iGP' 'VPM' 'CA1_Py' 'VM' 'dCA1' 'Sub' 'LDVL' 'VL' 'DG_a']
mvg_9_REW_bn_session_iNOGO_naive
0 : 0.32888304626983267 0.465506549439448 : ['M1_l' 'Pir' 'LO' 'Sub' 'eGP' 'Po' 'M2_1']
1 : 0.42369146817053743 0.5717893453687096 : ['AOL' 'AOD' 'M2' 'VO' 'Cg1' 'PrL' 'MO']
2 : 0.07948969301935221 0.10200347964243693 : ['V1_m' 'RSG' 'SuG' 'dCA1']
3 : 0.19189064481171705 0.24694630621084956 : ['AI' 'M1_m' 'IV2' 'DG_p' 'S1_bf' 'Cpu_1']
4 : 0.41264055870928934 0.520162547137573 : ['BIC' 'SubB' 'InWh' 'LPAG' 'Rt' 'iGP' 'VPM' 'CA1_Py' 'V

0 : 0.6109237445869772 0.7377991182009673 : ['M1_l' 'AI' 'M1_m' 'VO' 'PrL' 'dCA1']
1 : 0.5230486637781544 0.648354417627169 : ['Pir' 'LO' 'AOL' 'M2' 'SubB' 'S2' 'Cpu_1']
2 : 0.5815666900799195 0.6623622603726772 : ['IV2' 'DG_p' 'BIC' 'S1_bf_1' 'LaDL']
3 : 0.4851191175185384 0.5243206736656937 : ['S' 'CA1_Mol' 'S1_bf' 'Rt' 'Po' 'VL' 'DG_a']
4 : 0.5716903472274057 0.6260032424802505 : ['RSG' 'InWh' 'SuG' 'LPAG' 'VM' 'M2_1']
5 : 0.36382907204898746 0.41989728262220916 : ['V1_m' 'BMA' 'iGP' 'Sub']
6 : 0.30750293592900435 0.36323163354846977 : ['AOD' 'Cpu' 'LGP' 'eGP' 'GP']
7 : 0.6882974792102526 0.7934432253762077 : ['Cg1' 'MO' 'BLA' 'VPM' 'CA1_Py' 'VPL' 'LDVL']
mvg_7_PRE_bn_trial_None_expert


/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:246: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn("Affinity propagation did not converge, this model "
/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 : 0.8886578321881083 0.9376342616587117 : ['M1_l' 'AI' 'M1_m' 'iGP' 'Sub' 'VL']
1 : 0.843838849383565 1.0456028651722915 : ['Pir' 'S1_bf_1' 'M2_1']
2 : 0.8261948758080832 1.0253444692590645 : ['AOL' 'AOD' 'Cg1' 'LaDL']
3 : nan nan : ['V1_m']
4 : 0.9200311962471684 0.9817638018613063 : ['IV2' 'BIC' 'SubB']
5 : 0.9739045468763019 0.9332635751086362 : ['CA1_Mol' 'RSG' 'InWh' 'Cpu' 'S2']
6 : 0.8197847125589853 0.7521264077543584 : ['BLA' 'Rt' 'eGP']
7 : 0.8130809150258264 0.948533426580848 : ['LO' 'M2' 'SuG' 'LPAG' 'VM' 'GP' 'VPL']
8 : 0.8614103353958519 0.9189140073032045 : ['S' 'S1_bf' 'BMA' 'VPM' 'Cpu_1' 'LGP' 'LDVL' 'Po']
9 : 0.8958804876093887 1.1575312009903609 : ['VO' 'PrL' 'MO' 'DG_p' 'CA1_Py' 'dCA1' 'DG_a']
mvg_4_PRE_bn_trial_iGO_expert
0 : 0.8257380774192725 0.9768855983463912 : ['MO' 'dCA1' 'Po']
1 : 0.8559267165464824 1.0881334652251018 : ['IV2' 'S' 'VPM' 'GP']
2 : 0.878225533239129 0.9265089005514986 : ['CA1_Mol' 'LDVL']
3 : 0.7829358443324468 0.8446203470102003 : ['AOL' 'V1

/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 : 0.7752058998555811 0.8628688096869032 : ['M1_l' 'AI' 'M1_m' 'Rt' 'Sub' 'eGP' 'VL']
1 : 0.8120617032254721 0.9753860563364783 : ['Pir' 'S1_bf_1' 'M2_1']
2 : 0.8972831149103345 1.0532427870164345 : ['AOL' 'AOD' 'LaDL']
3 : nan nan : ['V1_m']
4 : 0.925250499372225 0.9622879445723705 : ['IV2' 'BIC' 'SubB']
5 : 0.9603917367047673 0.9607199174925278 : ['CA1_Mol' 'RSG' 'InWh' 'Cpu' 'S2']
6 : 0.8321133804395847 0.9369558899727406 : ['Cg1' 'LPAG' 'BLA' 'VPL']
7 : 0.940378158546135 1.101340313233197 : ['LO' 'M2' 'SuG' 'iGP' 'VM' 'GP']
8 : 0.8560643431716131 1.0495686321724842 : ['VO' 'PrL' 'MO' 'DG_p' 'CA1_Py' 'dCA1' 'DG_a']
9 : 0.8548918816457355 0.9428576231969311 : ['S' 'S1_bf' 'BMA' 'VPM' 'Cpu_1' 'LGP' 'LDVL' 'Po']
mvg_4_PRE_bn_trial_iNOGO_expert
0 : 0.8689041703887801 1.0046125995903714 : ['MO' 'dCA1' 'Po']
1 : 0.8836298215964836 1.1031559359256005 : ['IV2' 'S' 'VPM' 'GP']
2 : 0.8888338343317727 0.9362856427879307 : ['CA1_Mol' 'LDVL']
3 : 0.6697586535223744 0.7072383822397906 : ['AOL' '

0 : 0.515849127147215 0.7744945800747032 : ['AI' 'M1_m' 'LO' 'AOL' 'AOD' 'M2' 'InWh' 'BLA' 'Sub']
1 : 0.3574550885173785 0.4398287869766365 : ['Pir' 'PrL' 'MO' 'IV2' 'S' 'CA1_Mol' 'DG_p' 'BIC' 'SubB' 'S1_bf' 'VPM'
 'LGP' 'LDVL' 'Po' 'M2_1' 'DG_a']
2 : 0.179197236913393 0.19545683144999407 : ['VO' 'V1_m' 'RSG' 'SuG' 'Cpu' 'dCA1']
3 : 0.10433570630159657 0.14143935265517524 : ['M1_l' 'Cg1' 'LPAG' 'S1_bf_1' 'VM' 'LaDL']
4 : 0.23577504620516815 0.27391806508051053 : ['BMA' 'Rt' 'iGP' 'CA1_Py' 'S2' 'Cpu_1' 'eGP' 'GP' 'VPL' 'VL']
mvg_4_REW_bn_session_iGO_naive
0 : 0.4341734189987747 0.4815986088911737 : ['AI' 'M1_m' 'LO' 'AOD' 'Sub']
1 : 0.05459183816635738 0.09086595285047039 : ['AOL' 'M2' 'VO' 'SubB' 'S1_bf_1' 'BMA' 'Rt' 'CA1_Py' 'VM' 'dCA1' 'LaDL'
 'Po' 'M2_1']
2 : 0.0674862647479047 0.11977636592598193 : ['S' 'V1_m' 'LPAG' 'Cpu' 'BLA' 'iGP' 'VPM' 'S2' 'Cpu_1' 'eGP' 'GP' 'VPL'
 'VL']
3 : 0.09709854008007829 0.1771811044686218 : ['M1_l' 'Pir' 'Cg1' 'PrL' 'MO' 'IV2' 'CA1_Mol' 'DG_p' 'BIC' '

## 1.2 PCA exploration

# 2. Consistency
## 2.1. PCA consistency over mice
### 2.1.1. Angle-based consistency

Tasks
  * Explained variance by phase/session/mouse/trialType
     * Do not separate phases, its meaningless. Compute PCA for all timesteps, then see proj differences for phases
     * Implement HAC correction

  * Global PCA shifts vs session

Approaches:
  * Eval PCA over all data, select strongest components, plot components as function of cofound
  * Eval PCA for cofounds, compare PCA
  
**Plots**:
* Cosine-squared matrix $C^2_{ij} = (R^{1}_{ik}R^{2}_{jk})^2$, where $R^l$ is the PCA-transform
* Consistency metric $E = e^1_i e^2_j C^2_{ij}$, where $e^l$ are the eigenvalues

**Problem**:
The consistency metric $E$ has all necessary ingredients (angles, eigenvalues), but it is not mathematically clear that it behaves the desired way. Solid theory is required for this metric to be useful.

**Alternative approach**:
Try consistency metric $H(\frac{C^2_{ij}}{N})$. Should be great at measuring the sparsity of basis coupling. The challenge is to include eigenvalue priority into this metric, since spread of weak eigenvalues is not as relevant as spread of strong ones.

In [ ]:
coactivity.plot_pca_alignment_bymouse(dataDB, datatype='bn_session', trialType=None)

### 2.1.2. Eigenvalue-based consistency

* Let $x_1$, $x_2$ be some datasets
* Let $R_1$, $R_2$ be the corresponding PCA-transforms 
* Find total variances
    - $V_1 = \sum_i eig_i(x_1) = tr(cov(x_1)) = \sum_i cov_{ii}(x_1)$
    - $V_2 = \sum_i eig_i(x_2) = tr(cov(x_2)) = \sum_i cov_{ii}(x_2)$
* Find explained variances
    - $e_1 = eval(cov(x_1)) = diag(cov(R_1 x_1))$
    - $e_2 = eval(cov(x_2)) = diag(cov(R_2 x_2))$
* Find explained variances using wrong bases
    - $e_{12} = diag(cov(R_2 x_1))$
    - $e_{21} = diag(cov(R_1 x_2))$
* Find representation errors in explained variance ratios
    - $\epsilon_1 = \frac{\sum_i |e^1_i - e^{12}_i|}{2 V_1}$
    - $\epsilon_2 = \frac{\sum_i |e^2_i - e^{21}_i|}{2 V_2}$



* TODO: iter trialType=[hit, cr, all]
* TODO: iter perf=[naive,expert,all]

In [23]:
coactivity.plot_pca_consistency(dataDB, dropChannels=[21])

bn_trial_PRE
{(40, 48)}
{(40, 48)}
{(40, 48)}
{(40, 48)}
bn_trial_TEX
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
bn_trial_REW
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
bn_session_PRE
{(40, 48)}
{(40, 48)}
{(40, 48)}
{(40, 48)}
bn_session_TEX
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
bn_session_REW
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
raw_PRE
{(40, 48)}
{(40, 48)}
{(40, 48)}
{(40, 48)}
raw_TEX
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
raw_REW
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
['bn_session', 'bn_trial', 'raw'] ['PRE', 'REW', 'TEX']


In [24]:
coactivity.plot_pca_consistency(dataDB, dropChannels=[21], dropFirst=1)

bn_trial_PRE
{(40, 48)}
{(40, 48)}
{(40, 48)}
{(40, 48)}
bn_trial_TEX
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
bn_trial_REW
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
bn_session_PRE
{(40, 48)}
{(40, 48)}
{(40, 48)}
{(40, 48)}
bn_session_TEX
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
bn_session_REW
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
raw_PRE
{(40, 48)}
{(40, 48)}
{(40, 48)}
{(40, 48)}
raw_TEX
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
raw_REW
{(10, 48)}
{(10, 48)}
{(10, 48)}
{(10, 48)}
['bn_session', 'bn_trial', 'raw'] ['PRE', 'REW', 'TEX']


## 2.2. PCA consistency over phases
### 2.2.1 Angle-based consistency

In [ ]:
coactivity.plot_pca_alignment_byphase(dataDB, intervNames=['TEX', 'REW'], datatype='bn_trial', trialType=None)